In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit.aqua.components.ansatz import Ansatz, TwoLocalAnsatz, RY, RYRZ, SwapRZ

/Users/jul/Work/Qiskit/qiskit-aqua/qiskit/aqua/components/variational_forms/__init__.py:44: DeprecationWarning: This module is deprecated, VariationalForm is replaced by Ansatz, located in qiskit.aqua.components.ansatz
  from qiskit.aqua.components.variational_forms.variational_form import VariationalForm


# Ansatz Demo

This notebook presents new functionality introduced with Aqua's Ansatz class.

The goal of this refactor was to improve the flexibility of Ansatz-type objects, such as variational forms and feature maps, and to simplify the generation of new user-made Ansätze.

## The standard variational forms

Let's start off by playing with one of the standard variational forms, the RY variational form.
This Ansatz consists of layers of $Y$ rotations followed by layers of entanglements.

Before, all specifications of the Ansatz needed to be present in the initializer (except the parameters). 
Now, all attributes can be specified at any time. 

In [2]:
ry = RY()

Now an empty blueprint `RY` is created. For all attributes except the number of qubits, default arguments are provided. 
Therefore by specifying the number of qubits, we can already use the variational form.

In [3]:
ry.num_qubits = 3
ry

        ┌────────┐      ┌────────┐                ┌────────┐                »
q_0: |0>┤ Ry(θ0) ├─■──■─┤ Ry(θ3) ├───────────■──■─┤ Ry(θ6) ├───────────■──■─»
        ├────────┤ │  │ └────────┘┌────────┐ │  │ └────────┘┌────────┐ │  │ »
q_1: |0>┤ Ry(θ1) ├─■──┼─────■─────┤ Ry(θ4) ├─■──┼─────■─────┤ Ry(θ7) ├─■──┼─»
        ├────────┤    │     │     ├────────┤    │     │     ├────────┤    │ »
q_2: |0>┤ Ry(θ2) ├────■─────■─────┤ Ry(θ5) ├────■─────■─────┤ Ry(θ8) ├────■─»
        └────────┘                └────────┘                └────────┘      »
«     ┌────────┐           
«q_0: ┤ Ry(θ9) ├───────────
«     └────────┘┌─────────┐
«q_1: ────■─────┤ Ry(θ10) ├
«         │     ├─────────┤
«q_2: ────■─────┤ Ry(θ11) ├
«               └─────────┘

For convenience, printing the Ansatz object to the console draws the circuit diagram. 
Under the hood, the circuit is constructed using default parameters and transpiled using all standard gates of Terra.

### Modifying the Ansatz attributes

Per default, no barriers are inserted in between the rotation and entanglement layers, since this can influence the performance of transpiling and executing the circuit.
To insert barriers, simply set the attribute `insert_barriers` to `True`.
The circuit will then be rebuild on demand.

In [4]:
ry.insert_barriers = True
ry

        ┌────────┐ ░           ░ ┌────────┐ ░           ░ ┌────────┐ ░       »
q_0: |0>┤ Ry(θ0) ├─░──■──■─────░─┤ Ry(θ3) ├─░──■──■─────░─┤ Ry(θ6) ├─░──■──■─»
        ├────────┤ ░  │  │     ░ ├────────┤ ░  │  │     ░ ├────────┤ ░  │  │ »
q_1: |0>┤ Ry(θ1) ├─░──■──┼──■──░─┤ Ry(θ4) ├─░──■──┼──■──░─┤ Ry(θ7) ├─░──■──┼─»
        ├────────┤ ░     │  │  ░ ├────────┤ ░     │  │  ░ ├────────┤ ░     │ »
q_2: |0>┤ Ry(θ2) ├─░─────■──■──░─┤ Ry(θ5) ├─░─────■──■──░─┤ Ry(θ8) ├─░─────■─»
        └────────┘ ░           ░ └────────┘ ░           ░ └────────┘ ░       »
«         ░  ┌────────┐
«q_0: ────░──┤ Ry(θ9) ├
«         ░ ┌┴────────┤
«q_1: ─■──░─┤ Ry(θ10) ├
«      │  ░ ├─────────┤
«q_2: ─■──░─┤ Ry(θ11) ├
«         ░ └─────────┘

The default `depth`, i.e. how often rotation and entanglement layers are repeated, is set to 3. 
This attribute can be changed.

In [5]:
ry.depth = 1
ry

        ┌────────┐ ░           ░ ┌────────┐
q_0: |0>┤ Ry(θ0) ├─░──■──■─────░─┤ Ry(θ3) ├
        ├────────┤ ░  │  │     ░ ├────────┤
q_1: |0>┤ Ry(θ1) ├─░──■──┼──■──░─┤ Ry(θ4) ├
        ├────────┤ ░     │  │  ░ ├────────┤
q_2: |0>┤ Ry(θ2) ├─░─────■──■──░─┤ Ry(θ5) ├
        └────────┘ ░           ░ └────────┘

Similarly, the default entanglement strategy is `'full'`, meaning every qubit is entangled with every other qubit.
Other entanglement strategies that can be specified as string are `'linear'` or `'sca'`
(see the documentation for more detail about the latter).

In [6]:
ry.entanglement = 'linear'
ry

        ┌────────┐ ░        ░ ┌────────┐
q_0: |0>┤ Ry(θ0) ├─░──■─────░─┤ Ry(θ3) ├
        ├────────┤ ░  │     ░ ├────────┤
q_1: |0>┤ Ry(θ1) ├─░──■──■──░─┤ Ry(θ4) ├
        ├────────┤ ░     │  ░ ├────────┤
q_2: |0>┤ Ry(θ2) ├─░─────■──░─┤ Ry(θ5) ├
        └────────┘ ░        ░ └────────┘

The entanglement can also be specified via a list of qubits to entangle. 
Before, this has been specified using the `entangler_map` keyword (which is still supported but deprecated). 
Now, the this is also handled by the `entanglement` attribute. 

In [7]:
ry.entanglement = [[0, 2]]
ry

        ┌────────┐ ░     ░ ┌────────┐
q_0: |0>┤ Ry(θ0) ├─░──■──░─┤ Ry(θ3) ├
        ├────────┤ ░  │  ░ ├────────┤
q_1: |0>┤ Ry(θ1) ├─░──┼──░─┤ Ry(θ4) ├
        ├────────┤ ░  │  ░ ├────────┤
q_2: |0>┤ Ry(θ2) ├─░──■──░─┤ Ry(θ5) ├
        └────────┘ ░     ░ └────────┘

A new feature is the ability to provide a callable as entanglement, which takes as argument the index of the current entanglement block and returns the block-specific entangler map.

In [8]:
def entanglement_strategy(i):
    if i % 2 == 0:
        return [[0, 1], [1, 2], [2, 0]]
    return [[0, 1]]

ry.depth = 3
ry.entanglement = entanglement_strategy
ry

        ┌────────┐ ░           ░ ┌────────┐ ░     ░ ┌────────┐ ░           ░ »
q_0: |0>┤ Ry(θ0) ├─░──■─────■──░─┤ Ry(θ3) ├─░──■──░─┤ Ry(θ6) ├─░──■─────■──░─»
        ├────────┤ ░  │     │  ░ ├────────┤ ░  │  ░ ├────────┤ ░  │     │  ░ »
q_1: |0>┤ Ry(θ1) ├─░──■──■──┼──░─┤ Ry(θ4) ├─░──■──░─┤ Ry(θ7) ├─░──■──■──┼──░─»
        ├────────┤ ░     │  │  ░ ├────────┤ ░     ░ ├────────┤ ░     │  │  ░ »
q_2: |0>┤ Ry(θ2) ├─░─────■──■──░─┤ Ry(θ5) ├─░─────░─┤ Ry(θ8) ├─░─────■──■──░─»
        └────────┘ ░           ░ └────────┘ ░     ░ └────────┘ ░           ░ »
«      ┌────────┐
«q_0: ─┤ Ry(θ9) ├
«     ┌┴────────┤
«q_1: ┤ Ry(θ10) ├
«     ├─────────┤
«q_2: ┤ Ry(θ11) ├
«     └─────────┘

The `RY` variational form is implemented as specialized two-local Ansatz, which a general variational form with arbitrary single-qubit gates in the rotation layer and arbitrary two-qubit gates in the entanglement layer. We will introduce this class later on. 

Therefore, both the entanglement and rotation gates can be changed. The gates can be specified as string or circuit (in preparation: as gate type). Single or multiple gates can be set.

In [9]:
ry.entanglement_gates = 'cx'
ry.depth = 1
ry

        ┌────────┐ ░           ┌───┐ ░ ┌────────┐
q_0: |0>┤ Ry(θ0) ├─░───■───────┤ X ├─░─┤ Ry(θ3) ├
        ├────────┤ ░ ┌─┴─┐     └─┬─┘ ░ ├────────┤
q_1: |0>┤ Ry(θ1) ├─░─┤ X ├──■────┼───░─┤ Ry(θ4) ├
        ├────────┤ ░ └───┘┌─┴─┐  │   ░ ├────────┤
q_2: |0>┤ Ry(θ2) ├─░──────┤ X ├──■───░─┤ Ry(θ5) ├
        └────────┘ ░      └───┘      ░ └────────┘

In [10]:
ry.entanglement_gates = ['cx', 'cry']
ry

        ┌────────┐ ░                               ┌───┐┌────────┐ ░ ┌────────┐
q_0: |0>┤ Ry(θ0) ├─░───■──────■────────────────────┤ X ├┤ Ry(θ5) ├─░─┤ Ry(θ6) ├
        ├────────┤ ░ ┌─┴─┐┌───┴────┐               └─┬─┘└───┬────┘ ░ ├────────┤
q_1: |0>┤ Ry(θ1) ├─░─┤ X ├┤ Ry(θ3) ├──■──────■───────┼──────┼──────░─┤ Ry(θ7) ├
        ├────────┤ ░ └───┘└────────┘┌─┴─┐┌───┴────┐  │      │      ░ ├────────┤
q_2: |0>┤ Ry(θ2) ├─░────────────────┤ X ├┤ Ry(θ4) ├──■──────■──────░─┤ Ry(θ8) ├
        └────────┘ ░                └───┘└────────┘                ░ └────────┘

To prepare an individual entanglement gate just create a 2-qubit circuit with `qiskit.circuit.Parameter` objects as parameters.

In [11]:
from qiskit.circuit import Parameter

my_entanglement = QuantumCircuit(2)
angle = Parameter('α')
my_entanglement.rxx(angle, 0, 1)
my_entanglement.x(1)
my_entanglement.ry(angle, 0)

ry.entanglement_gates = my_entanglement
ry

        ┌────────┐ ░ ┌──────────┐┌────────┐                      ┌──────────┐»
q_0: |0>┤ Ry(θ0) ├─░─┤0         ├┤ Ry(θ3) ├──────────────────────┤1         ├»
        ├────────┤ ░ │  Rxx(θ3) │└─┬───┬──┘┌──────────┐┌────────┐│          │»
q_1: |0>┤ Ry(θ1) ├─░─┤1         ├──┤ X ├───┤0         ├┤ Ry(θ4) ├┤  Rxx(θ5) ├»
        ├────────┤ ░ └──────────┘  └───┘   │  Rxx(θ4) │└─┬───┬──┘│          │»
q_2: |0>┤ Ry(θ2) ├─░───────────────────────┤1         ├──┤ X ├───┤0         ├»
        └────────┘ ░                       └──────────┘  └───┘   └──────────┘»
«       ┌───┐    ░ ┌────────┐
«q_0: ──┤ X ├────░─┤ Ry(θ6) ├
«       └───┘    ░ ├────────┤
«q_1: ───────────░─┤ Ry(θ7) ├
«     ┌────────┐ ░ ├────────┤
«q_2: ┤ Ry(θ5) ├─░─┤ Ry(θ8) ├
«     └────────┘ ░ └────────┘

The rotation gates can be set in the same fashion.

In [12]:
ry.entanglement_gates = 'crx'
ry.rotation_gates = ['z', 'ry', 'h']
ry

        ┌───┐┌────────┐┌───┐ ░                     ┌────────┐ ░ ┌───┐┌────────┐»
q_0: |0>┤ Z ├┤ Ry(θ0) ├┤ H ├─░─────■───────────────┤ Rx(θ5) ├─░─┤ Z ├┤ Ry(θ6) ├»
        ├───┤├────────┤├───┤ ░ ┌───┴────┐          └───┬────┘ ░ ├───┤├────────┤»
q_1: |0>┤ Z ├┤ Ry(θ1) ├┤ H ├─░─┤ Rx(θ3) ├────■─────────┼──────░─┤ Z ├┤ Ry(θ7) ├»
        ├───┤├────────┤├───┤ ░ └────────┘┌───┴────┐    │      ░ ├───┤├────────┤»
q_2: |0>┤ Z ├┤ Ry(θ2) ├┤ H ├─░───────────┤ Rx(θ4) ├────■──────░─┤ Z ├┤ Ry(θ8) ├»
        └───┘└────────┘└───┘ ░           └────────┘           ░ └───┘└────────┘»
«     ┌───┐
«q_0: ┤ H ├
«     ├───┤
«q_1: ┤ H ├
«     ├───┤
«q_2: ┤ H ├
«     └───┘

### Parameter updates

The parameters of the Ansatz are per default `qiskit.circuit.Parameter` objects numbered from `0` to `num_parameter - 1` with a certain parameter prefix (per default `'θ'`).

In [13]:
print('The Ansatz has', ry.num_parameters, 'parameters.')
print('Namely:', ry.parameters)

The Ansatz has 9 parameters.
Namely: [Parameter(θ0), Parameter(θ1), Parameter(θ2), Parameter(θ3), Parameter(θ4), Parameter(θ5), Parameter(θ6), Parameter(θ7), Parameter(θ8)]


To set new parameters, use the attribute `parameters`. 
This can be set either to a list of numbers or to another list of parameters.

In [14]:
ry.parameters = list(range(ry.num_parameters))
ry

        ┌───┐┌───────┐┌───┐ ░                   ┌───────┐ ░ ┌───┐┌───────┐┌───┐
q_0: |0>┤ Z ├┤ Ry(0) ├┤ H ├─░─────■─────────────┤ Rx(5) ├─░─┤ Z ├┤ Ry(6) ├┤ H ├
        ├───┤├───────┤├───┤ ░ ┌───┴───┐         └───┬───┘ ░ ├───┤├───────┤├───┤
q_1: |0>┤ Z ├┤ Ry(1) ├┤ H ├─░─┤ Rx(3) ├────■────────┼─────░─┤ Z ├┤ Ry(7) ├┤ H ├
        ├───┤├───────┤├───┤ ░ └───────┘┌───┴───┐    │     ░ ├───┤├───────┤├───┤
q_2: |0>┤ Z ├┤ Ry(2) ├┤ H ├─░──────────┤ Rx(4) ├────■─────░─┤ Z ├┤ Ry(8) ├┤ H ├
        └───┘└───────┘└───┘ ░          └───────┘          ░ └───┘└───────┘└───┘

In [15]:
ry.parameters = [Parameter('χ' + str(i)) for i in range(ry.num_parameters)]
ry

        ┌───┐┌────────┐┌───┐ ░                     ┌────────┐ ░ ┌───┐┌────────┐»
q_0: |0>┤ Z ├┤ Ry(χ0) ├┤ H ├─░─────■───────────────┤ Rx(χ5) ├─░─┤ Z ├┤ Ry(χ6) ├»
        ├───┤├────────┤├───┤ ░ ┌───┴────┐          └───┬────┘ ░ ├───┤├────────┤»
q_1: |0>┤ Z ├┤ Ry(χ1) ├┤ H ├─░─┤ Rx(χ3) ├────■─────────┼──────░─┤ Z ├┤ Ry(χ7) ├»
        ├───┤├────────┤├───┤ ░ └────────┘┌───┴────┐    │      ░ ├───┤├────────┤»
q_2: |0>┤ Z ├┤ Ry(χ2) ├┤ H ├─░───────────┤ Rx(χ4) ├────■──────░─┤ Z ├┤ Ry(χ8) ├»
        └───┘└────────┘└───┘ ░           └────────┘           ░ └───┘└────────┘»
«     ┌───┐
«q_0: ┤ H ├
«     ├───┤
«q_1: ┤ H ├
«     ├───┤
«q_2: ┤ H ├
«     └───┘

### Creating circuits and instructions

To obtain the underlying circuit of the Ansatz, call `to_circuit`. 
Note that the diagram will show the blocks the Ansatz is made up of.

In [16]:
circuit = ry.to_circuit()
circuit.draw()

┌─────────────────┐ ░ ┌─────────────────┐ ░ ┌─────────────────┐
q_0: |0>┤0                ├─░─┤0                ├─░─┤0                ├
        │                 │ ░ │                 │ ░ │                 │
q_1: |0>┤1 Rot0(χ0,χ1,χ2) ├─░─┤1 Ent0(χ3,χ4,χ5) ├─░─┤1 Rot1(χ6,χ7,χ8) ├
        │                 │ ░ │                 │ ░ │                 │
q_2: |0>┤2                ├─░─┤2                ├─░─┤2                ├
        └─────────────────┘ ░ └─────────────────┘ ░ └─────────────────┘

To unveil the constituents decompose or transpile the circuit.

In [17]:
circuit.decompose().draw()

┌───┐┌────────┐┌───┐ ░                     ┌────────┐ ░ ┌───┐┌────────┐»
q_0: |0>┤ Z ├┤ Ry(χ0) ├┤ H ├─░─────■───────────────┤ Rx(χ5) ├─░─┤ Z ├┤ Ry(χ6) ├»
        ├───┤├────────┤├───┤ ░ ┌───┴────┐          └───┬────┘ ░ ├───┤├────────┤»
q_1: |0>┤ Z ├┤ Ry(χ1) ├┤ H ├─░─┤ Rx(χ3) ├────■─────────┼──────░─┤ Z ├┤ Ry(χ7) ├»
        ├───┤├────────┤├───┤ ░ └────────┘┌───┴────┐    │      ░ ├───┤├────────┤»
q_2: |0>┤ Z ├┤ Ry(χ2) ├┤ H ├─░───────────┤ Rx(χ4) ├────■──────░─┤ Z ├┤ Ry(χ8) ├»
        └───┘└────────┘└───┘ ░           └────────┘           ░ └───┘└────────┘»
«     ┌───┐
«q_0: ┤ H ├
«     ├───┤
«q_1: ┤ H ├
«     ├───┤
«q_2: ┤ H ├
«     └───┘

Instead of creating a circuit the Ansatz can also be converted to an `Instruction` or `Gate` object.
The `Gate` representation is important because it allows access to functionality such as `control()` and `inverse()` or `repeat()`.

Note however, that a Ansatz can only be converted to a `Gate` if it has no non-unitary operations, such as measurements, barriers or resets.

In [18]:
instruction = ry.to_instruction()
ry.insert_barriers = False
gate = ry.to_gate()
ry.insert_barriers = True

In [19]:
circuit = QuantumCircuit(4)
circuit.append(gate.control(), [0, 1, 2, 3])
circuit.append(gate.inverse(), [1, 2, 3])
circuit.draw()

»
q_0: |0>─────────────────────■─────────────────────»
        ┌───────────────────┴┼────────────────────┐»
q_1: |0>┤0                   │                    ├»
        │                                         │»
q_2: |0>┤1 Circuit188(χ0,χ1,χ2,χ3,χ4,χ5,χ6,χ7,χ8) ├»
        │                                         │»
q_3: |0>┤2                                        ├»
        └─────────────────────────────────────────┘»
«                                                   
«q_0: ──────────────────────────────────────────────
«     ┌────────────────────────────────────────────┐
«q_1: ┤0                                           ├
«     │                                            │
«q_2: ┤1 Circuit188_dg(χ0,χ1,χ2,χ3,χ4,χ5,χ6,χ7,χ8) ├
«     │                                            │
«q_3: ┤2                                           ├
«     └────────────────────────────────────────────┘

Also available are the `RYRZ`, `SwapRZ` and `UCCSD` Ansatz (latter is shown in an example below).

In [20]:
RYRZ(2)

        ┌────────┐┌────────┐   ┌────────┐┌────────┐    ┌────────┐ ┌────────┐   »
q_0: |0>┤ Ry(θ0) ├┤ Rz(θ1) ├─■─┤ Ry(θ4) ├┤ Rz(θ5) ├─■──┤ Ry(θ8) ├─┤ Rz(θ9) ├─■─»
        ├────────┤├────────┤ │ ├────────┤├────────┤ │ ┌┴────────┤┌┴────────┤ │ »
q_1: |0>┤ Ry(θ2) ├┤ Rz(θ3) ├─■─┤ Ry(θ6) ├┤ Rz(θ7) ├─■─┤ Ry(θ10) ├┤ Rz(θ11) ├─■─»
        └────────┘└────────┘   └────────┘└────────┘   └─────────┘└─────────┘   »
«     ┌─────────┐┌─────────┐
«q_0: ┤ Ry(θ12) ├┤ Rz(θ13) ├
«     ├─────────┤├─────────┤
«q_1: ┤ Ry(θ14) ├┤ Rz(θ15) ├
«     └─────────┘└─────────┘

In [21]:
SwapRZ(3, depth=1, insert_barriers=True)

        ┌────────┐ ░ ┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
q_0: |0>┤ Rz(θ0) ├─░─┤0         ├┤0         ├┤0         ├┤0         ├»
        ├────────┤ ░ │  Rxx(θ3) ││  Ryy(θ3) ││          ││          │»
q_1: |0>┤ Rz(θ1) ├─░─┤1         ├┤1         ├┤  Rxx(θ4) ├┤  Ryy(θ4) ├»
        ├────────┤ ░ └──────────┘└──────────┘│          ││          │»
q_2: |0>┤ Rz(θ2) ├─░─────────────────────────┤1         ├┤1         ├»
        └────────┘ ░                         └──────────┘└──────────┘»
«                              ░ ┌────────┐
«q_0: ─────────────────────────░─┤ Rz(θ6) ├
«     ┌──────────┐┌──────────┐ ░ ├────────┤
«q_1: ┤0         ├┤0         ├─░─┤ Rz(θ7) ├
«     │  Rxx(θ5) ││  Ryy(θ5) │ ░ ├────────┤
«q_2: ┤1         ├┤1         ├─░─┤ Rz(θ8) ├
«     └──────────┘└──────────┘ ░ └────────┘

## Backward compatibility

Backward compatibility is maintained mainly by providing a `construct_circuit` method.

In [22]:
ry = RY(3, depth=1, insert_barriers=True)

In [23]:
bound_circuit = ry.construct_circuit(params=list(range(ry.num_parameters)))
bound_circuit.decompose().draw()

┌───────┐ ░           ░ ┌───────┐
q_0: |0>┤ Ry(0) ├─░──■──■─────░─┤ Ry(3) ├
        ├───────┤ ░  │  │     ░ ├───────┤
q_1: |0>┤ Ry(1) ├─░──■──┼──■──░─┤ Ry(4) ├
        ├───────┤ ░     │  │  ░ ├───────┤
q_2: |0>┤ Ry(2) ├─░─────■──■──░─┤ Ry(5) ├
        └───────┘ ░           ░ └───────┘

### Running the VQE

Current algorithms, such as the VQE do not need any changes.

In [24]:
from qiskit import BasicAer
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.components.optimizers import SLSQP

reference_energy = -1.8572750

pauli_dict = {
    'paulis': [{"coeff": {"imag": 0.0, "real": -1.052373245772859}, "label": "II"},
               {"coeff": {"imag": 0.0, "real": 0.39793742484318045}, "label": "IZ"},
               {"coeff": {"imag": 0.0, "real": -0.39793742484318045}, "label": "ZI"},
               {"coeff": {"imag": 0.0, "real": -0.01128010425623538}, "label": "ZZ"},
               {"coeff": {"imag": 0.0, "real": 0.18093119978423156}, "label": "XX"}
              ]
              }

qubit_op = WeightedPauliOperator.from_dict(pauli_dict)
ry = RY(2)
optimizer = SLSQP()
backend = BasicAer.get_backend('statevector_simulator')

vqe = VQE(qubit_op, ry, optimizer)
result = vqe.run(backend)

In [25]:
print('Energy:', result['energy'])
print('Expected:', reference_energy)

Energy: -1.8572749546998735
Expected: -1.857275


In [26]:
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import HDF5Driver
from qiskit.chemistry.core import Hamiltonian, QubitMappingType


reference_energy = -1.1373060356951838

driver = HDF5Driver('test_driver_hdf5.hdf5')
qmolecule = driver.run()
operator = Hamiltonian(qubit_mapping=QubitMappingType.PARITY, two_qubit_reduction=True)
qubit_op, _ = operator.run(qmolecule)

optimizer = SLSQP(maxiter=100)
initial_state = HartreeFock(qubit_op.num_qubits,
                            operator.molecule_info['num_orbitals'],
                            operator.molecule_info['num_particles'],
                            qubit_mapping=operator._qubit_mapping,
                            two_qubit_reduction=operator._two_qubit_reduction)
var_form = UCCSD(qubit_op.num_qubits, depth=1,
                 num_orbitals=operator.molecule_info['num_orbitals'],
                 num_particles=operator.molecule_info['num_particles'],
                 initial_state=initial_state,
                 qubit_mapping=operator._qubit_mapping,
                 two_qubit_reduction=operator._two_qubit_reduction)
algo = VQE(qubit_op, var_form, optimizer)
result = algo.run(BasicAer.get_backend('statevector_simulator'))
_, result = operator.process_algorithm_result(result)

/Users/jul/Work/Qiskit/qiskit-aqua/qiskit/aqua/components/ansatz/ansatz.py:481: UserWarning: This private class member is deprecated and will be removed. Use the property num_parameters instead.
  warnings.warn('This private class member is deprecated and will be removed. '
/Users/jul/Work/Qiskit/qiskit-aqua/qiskit/aqua/components/ansatz/ansatz.py:474: UserWarning: This private class member is deprecated and will be removed. Use the property num_parameters instead.
  warnings.warn('This private class member is deprecated and will be removed. '


In [27]:
print('Energy:', result['energy'])
print('Expected:', reference_energy)

Energy: -1.1373060356950961
Expected: -1.1373060356951838


## Creating special two-local Ansätze

To set up a two-local Ansatz, the class `TwoLocalAnsatz` can be used. 
This class has essentially the same arguments as `RY` or `RYRZ` but requires arguments for the rotation and entanglement gates.

In [28]:
from qiskit.aqua.components.ansatz import TwoLocalAnsatz

In [29]:
ansatz = TwoLocalAnsatz(num_qubits=3, depth=1, rotation_gates=['h', 'rz', 'h'], entanglement_gates='crx', insert_barriers=True)
ansatz

        ┌───┐┌────────┐┌───┐ ░                                ░ ┌───┐┌────────┐»
q_0: |0>┤ H ├┤ Rz(θ0) ├┤ H ├─░─────■─────────■────────────────░─┤ H ├┤ Rz(θ6) ├»
        ├───┤├────────┤├───┤ ░ ┌───┴────┐    │                ░ ├───┤├────────┤»
q_1: |0>┤ H ├┤ Rz(θ1) ├┤ H ├─░─┤ Rx(θ3) ├────┼─────────■──────░─┤ H ├┤ Rz(θ7) ├»
        ├───┤├────────┤├───┤ ░ └────────┘┌───┴────┐┌───┴────┐ ░ ├───┤├────────┤»
q_2: |0>┤ H ├┤ Rz(θ2) ├┤ H ├─░───────────┤ Rx(θ4) ├┤ Rx(θ5) ├─░─┤ H ├┤ Rz(θ8) ├»
        └───┘└────────┘└───┘ ░           └────────┘└────────┘ ░ └───┘└────────┘»
«     ┌───┐
«q_0: ┤ H ├
«     ├───┤
«q_1: ┤ H ├
«     ├───┤
«q_2: ┤ H ├
«     └───┘

## Full control using the Ansatz class

Conceptually, the `Ansatz` base class takes as input a number of blocks in form of a circuit or instruction and a number of repetitions and repeats the input blocks.

In [30]:
num_qubits = 3
block0 = QuantumCircuit(num_qubits, name='block0')
block1 = QuantumCircuit(num_qubits, name='block1')

ansatz = Ansatz([block0, block1])
ansatz.to_circuit().draw()  # not using print(ansatz) since the tranpiling exposes the empty circuits

┌─────────┐┌─────────┐
q_0: |0>┤0        ├┤0        ├
        │         ││         │
q_1: |0>┤1 Block0 ├┤1 Block1 ├
        │         ││         │
q_2: |0>┤2        ├┤2        ├
        └─────────┘└─────────┘

The repetitions can be provided either as integer or as list of indices.

In [31]:
ansatz = Ansatz([block0, block1], reps=3)
ansatz.to_circuit().draw()

┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐
q_0: |0>┤0        ├┤0        ├┤0        ├┤0        ├┤0        ├┤0        ├
        │         ││         ││         ││         ││         ││         │
q_1: |0>┤1 Block0 ├┤1 Block1 ├┤1 Block0 ├┤1 Block1 ├┤1 Block0 ├┤1 Block1 ├
        │         ││         ││         ││         ││         ││         │
q_2: |0>┤2        ├┤2        ├┤2        ├┤2        ├┤2        ├┤2        ├
        └─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘

In [32]:
ansatz.reps = [0, 1, 1, 0]  # equivalent to Ansatz([block0, block1, block1, block0])
ansatz.to_circuit().draw()

┌─────────┐┌─────────┐┌─────────┐┌─────────┐
q_0: |0>┤0        ├┤0        ├┤0        ├┤0        ├
        │         ││         ││         ││         │
q_1: |0>┤1 Block0 ├┤1 Block1 ├┤1 Block1 ├┤1 Block0 ├
        │         ││         ││         ││         │
q_2: |0>┤2        ├┤2        ├┤2        ├┤2        ├
        └─────────┘└─────────┘└─────────┘└─────────┘

If the blocks are parameterized, the parameters are overwritten by default. The reason is that a repeated block probably should have a new parameter.


In [33]:
num_qubits = 2
block0 = QuantumCircuit(num_qubits, name='block0')
block0.rx(Parameter('X'), 0)
block1 = QuantumCircuit(num_qubits, name='block1')
block1.ry(Parameter('Y'), 1)

In [34]:
ansatz = Ansatz([block0, block1], reps=2, insert_barriers=True)
print('Parameters:', ansatz.parameters)
ansatz

Parameters: [Parameter(θ0), Parameter(θ1), Parameter(θ2), Parameter(θ3)]


        ┌────────┐ ░            ░ ┌────────┐ ░           
q_0: |0>┤ Rx(θ0) ├─░────────────░─┤ Rx(θ2) ├─░───────────
        └────────┘ ░ ┌────────┐ ░ └────────┘ ░ ┌────────┐
q_1: |0>───────────░─┤ Ry(θ1) ├─░────────────░─┤ Ry(θ3) ├
                   ░ └────────┘ ░            ░ └────────┘

This behaviour can be turned off using `overwrite_block_parameters`.

In [35]:
ansatz = Ansatz([block0, block1], reps=2, insert_barriers=True, overwrite_block_parameters=False)
print('Parameters:', ansatz.parameters)
ansatz

Parameters: [Parameter(X), Parameter(Y)]


        ┌───────┐ ░           ░ ┌───────┐ ░          
q_0: |0>┤ Rx(X) ├─░───────────░─┤ Rx(X) ├─░──────────
        └───────┘ ░ ┌───────┐ ░ └───────┘ ░ ┌───────┐
q_1: |0>──────────░─┤ Ry(Y) ├─░───────────░─┤ Ry(Y) ├
                  ░ └───────┘ ░           ░ └───────┘

If parameters should be bound, this can be achieved by providing a individual list of block parameters to the Ansatz.

Say the first 3 parameters should be the same and the last one different.

In [36]:
first, last = Parameter('A'), Parameter('B')
blockwise_parameters = 3 * [[first]] + [[last]]

ansatz = Ansatz([block0, block1], reps=2, insert_barriers=True, overwrite_block_parameters=blockwise_parameters)
print('Parameters:', ansatz.parameters)
ansatz

Parameters: [Parameter(A), Parameter(B)]


        ┌───────┐ ░           ░ ┌───────┐ ░          
q_0: |0>┤ Rx(A) ├─░───────────░─┤ Rx(A) ├─░──────────
        └───────┘ ░ ┌───────┐ ░ └───────┘ ░ ┌───────┐
q_1: |0>──────────░─┤ Ry(A) ├─░───────────░─┤ Ry(B) ├
                  ░ └───────┘ ░           ░ └───────┘

Using the `blockwise_parameters` attribute, the parameter bindings can be changed at any point.

In [37]:
ansatz.blockwise_parameters = [[Parameter('p' + str(i))] for i in range(4)]
ansatz

        ┌────────┐ ░            ░ ┌────────┐ ░           
q_0: |0>┤ Rx(p0) ├─░────────────░─┤ Rx(p2) ├─░───────────
        └────────┘ ░ ┌────────┐ ░ └────────┘ ░ ┌────────┐
q_1: |0>───────────░─┤ Ry(p1) ├─░────────────░─┤ Ry(p3) ├
                   ░ └────────┘ ░            ░ └────────┘